In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Set style parameters
plt.style.use('seaborn')
blue_palette = ['#1f77b4', '#7aa6c2', '#aec7e8', '#c7d9e8', '#e1e7ed']
sns.set_palette(blue_palette)

# Define the correct data path
data_path = os.path.join('Data', 'Combined_Data')

# Verify files exist in the Combined_Data directory
required_files = ['combined_accidents.csv', 'combined_casualties.csv', 'combined_vehicles.csv']
for file in required_files:
    file_path = os.path.join(data_path, file)
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Missing required file: {file_path}")

# Read the data
print("Reading data files...")
accidents_df = pd.read_csv(os.path.join(data_path, 'combined_accidents.csv'))
casualties_df = pd.read_csv(os.path.join(data_path, 'combined_casualties.csv'))
vehicles_df = pd.read_csv(os.path.join(data_path, 'combined_vehicles.csv'))

# Convert Date column to datetime with UK format (day first)
accidents_df['Date'] = pd.to_datetime(accidents_df['Date'], dayfirst=True)
accidents_df['Year'] = accidents_df['Date'].dt.year
accidents_df['Month'] = accidents_df['Date'].dt.month

# Define time parsing function
def parse_time(time_str):
    """Parse time string to hour, handling 'Unknown' and invalid values."""
    if pd.isna(time_str) or time_str == 'Unknown':
        return np.nan
    try:
        # For HH:MM format
        if ':' in str(time_str):
            return int(str(time_str).split(':')[0])
        return np.nan
    except:
        return np.nan

# Convert Time to Hour with robust parsing
print("Processing time data...")
accidents_df['Hour'] = accidents_df['Time'].apply(parse_time)

# Create output directory
os.makedirs('visualizations', exist_ok=True)
print("Created visualizations directory")

# Initialize list to store all plot divs
plot_divs = []

# Calculate some summary statistics for our analysis
total_accidents = len(accidents_df)
total_casualties = len(casualties_df)
total_vehicles = len(vehicles_df)
severity_counts = accidents_df['Accident_Severity'].value_counts()
yearly_counts = accidents_df['Year'].value_counts().sort_index()

# 1. Interactive Accident Severity Trend
print("Generating severity trend visualization...")
severity_by_year = accidents_df.groupby(['Year', 'Accident_Severity']).size().unstack()
fig_severity = go.Figure()
for severity in severity_by_year.columns:
    fig_severity.add_trace(go.Bar(
        x=severity_by_year.index,
        y=severity_by_year[severity],
        name=severity,
        marker_color=blue_palette[list(severity_by_year.columns).index(severity)]
    ))
fig_severity.update_layout(
    title='Accident Severity Distribution by Year',
    barmode='stack',
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font=dict(color='#444444'),
    xaxis_title="Year",
    yaxis_title="Number of Accidents",
    showlegend=True,
    legend_title="Severity Level",
    height=600
)
plot_divs.append(fig_severity.to_html(full_html=False, include_plotlyjs='cdn'))



C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_5200\2308610849.py:12: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


Reading data files...
Processing time data...
Created visualizations directory
Generating severity trend visualization...


In [2]:
# 2. Time of Day Heatmap
print("Generating time of day heatmap...")
accidents_with_time = accidents_df.dropna(subset=['Hour'])
hourly_severity = pd.crosstab(accidents_with_time['Hour'], accidents_with_time['Accident_Severity'])
fig_time = px.imshow(hourly_severity,
                    color_continuous_scale='Blues',
                    title='Accident Severity by Hour of Day',
                    labels=dict(x="Severity Level", y="Hour of Day", color="Number of Accidents"))
fig_time.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    height=600
)
plot_divs.append(fig_time.to_html(full_html=False, include_plotlyjs=False))

# 3. Interactive Age Distribution
print("Generating age distribution visualization...")
fig_age = px.violin(casualties_df,
                   x='Casualty_Severity',
                   y='Age_of_Casualty',
                   color='Casualty_Severity',
                   color_discrete_sequence=blue_palette,
                   box=True,
                   title='Age Distribution by Casualty Severity',
                   labels={'Casualty_Severity': 'Severity Level',
                          'Age_of_Casualty': 'Age'})
fig_age.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    height=600
)
plot_divs.append(fig_age.to_html(full_html=False, include_plotlyjs=False))

# 4. Weather Impact Sunburst
print("Generating weather impact visualization...")
fig_weather = px.sunburst(
    accidents_df,
    path=['Weather_Conditions', 'Accident_Severity'],
    color_discrete_sequence=blue_palette,
    title='Weather Conditions Impact on Accident Severity'
)
fig_weather.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    height=600
)
plot_divs.append(fig_weather.to_html(full_html=False, include_plotlyjs=False))

# 5. Vehicle Type Tree Map
print("Generating vehicle type visualization...")
vehicle_counts = vehicles_df['Vehicle_Type'].value_counts().reset_index()
vehicle_counts.columns = ['Vehicle_Type', 'Count']
fig_vehicles = px.treemap(
    vehicle_counts,
    path=['Vehicle_Type'],
    values='Count',
    color_discrete_sequence=blue_palette,
    title='Distribution of Vehicle Types in Accidents'
)
fig_vehicles.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    height=600
)
plot_divs.append(fig_vehicles.to_html(full_html=False, include_plotlyjs=False))

# 6. Road Speed Analysis
print("Generating road speed analysis...")
fig_road = px.box(accidents_df,
                 x='Road_Type',
                 y='Speed_limit',
                 color='Road_Type',
                 color_discrete_sequence=blue_palette,
                 title='Speed Limits by Road Type',
                 labels={'Road_Type': 'Type of Road',
                        'Speed_limit': 'Speed Limit (mph)'})
fig_road.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    xaxis={'tickangle': 45},
    height=600
)
plot_divs.append(fig_road.to_html(full_html=False, include_plotlyjs=False))

# 7. Casualty Demographics Bubble Plot
print("Generating casualty demographics visualization...")
casualty_demographics = casualties_df.groupby(
    ['Age_Band_of_Casualty', 'Sex_of_Casualty', 'Casualty_Severity']
).size().reset_index(name='count')

fig_demographics = px.scatter(
    casualty_demographics,
    x='Age_Band_of_Casualty',
    y='Casualty_Severity',
    size='count',
    color='Sex_of_Casualty',
    color_discrete_sequence=blue_palette,
    hover_data=['count'],
    title='Casualty Demographics Analysis',
    labels={'Age_Band_of_Casualty': 'Age Band',
            'Casualty_Severity': 'Severity Level',
            'count': 'Number of Casualties',
            'Sex_of_Casualty': 'Gender'}
)
fig_demographics.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    xaxis={'tickangle': 45},
    height=600
)
plot_divs.append(fig_demographics.to_html(full_html=False, include_plotlyjs=False))

# Calculate additional statistics for our analysis
weather_impact = pd.crosstab(accidents_df['Weather_Conditions'], 
                           accidents_df['Accident_Severity'], 
                           normalize='index') * 100

casualty_age_stats = casualties_df.groupby('Casualty_Severity')['Age_of_Casualty'].agg(['mean', 'median', 'std']).round(1)

# Begin HTML generation with comprehensive analysis
print("Generating HTML report...")

Generating time of day heatmap...
Generating age distribution visualization...
Generating weather impact visualization...


c:\Users\Wolfrank\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:1985: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Generating vehicle type visualization...
Generating road speed analysis...
Generating casualty demographics visualization...


c:\Users\Wolfrank\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:1985: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Generating HTML report...


In [3]:
# Generate comprehensive HTML report
html_content = f'''
<!DOCTYPE html>
<html>
<head>
    <title>UK Traffic Accident Analysis (2015-2018)</title>
    <style>
        body {{
            font-family: 'Helvetica Neue', Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f5f7fa;
            color: #2c3e50;
            line-height: 1.6;
        }}
        .navbar {{
            background-color: #1f77b4;
            padding: 15px 20px;
            position: fixed;
            width: 100%;
            top: 0;
            z-index: 1000;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        }}
        .navbar a {{
            color: white;
            text-decoration: none;
            margin-right: 20px;
            font-size: 14px;
            transition: opacity 0.3s;
        }}
        .navbar a:hover {{
            opacity: 0.8;
        }}
        .container {{
            max-width: 1200px;
            margin: 80px auto 20px auto;
            padding: 20px;
        }}
        .intro-section {{
            background-color: white;
            padding: 30px;
            border-radius: 10px;
            margin-bottom: 30px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }}
        .dataset-section {{
            background-color: #f8f9fa;
            padding: 20px;
            border-radius: 8px;
            margin: 20px 0;
        }}
        .visualization-section {{
            background-color: white;
            margin-bottom: 30px;
            padding: 30px;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }}
        h1 {{
            color: #1f77b4;
            text-align: center;
            margin-bottom: 40px;
            font-size: 2.5em;
        }}
        h2 {{
            color: #2c3e50;
            border-bottom: 2px solid #eef2f5;
            padding-bottom: 10px;
            margin-top: 40px;
            font-size: 1.8em;
        }}
        h3 {{
            color: #34495e;
            margin-top: 25px;
            font-size: 1.4em;
        }}
        .key-stat {{
            background-color: #f8f9fa;
            padding: 15px;
            border-radius: 5px;
            margin: 10px 0;
            border-left: 4px solid #1f77b4;
        }}
        .insight-box {{
            background-color: #e8f4f8;
            padding: 20px;
            border-radius: 5px;
            margin: 15px 0;
        }}
        .data-table {{
            width: 100%;
            border-collapse: collapse;
            margin: 15px 0;
        }}
        .data-table th, .data-table td {{
            padding: 12px;
            border: 1px solid #dee2e6;
            text-align: left;
        }}
        .data-table th {{
            background-color: #f8f9fa;
        }}
        code {{
            background-color: #f8f9fa;
            padding: 2px 5px;
            border-radius: 3px;
            font-family: 'Courier New', monospace;
        }}
    </style>
</head>
<body>
    <div class="navbar">
        <a href="#overview">Overview</a>
        <a href="#datasets">Datasets</a>
        <a href="#severity">Severity Analysis</a>
        <a href="#temporal">Temporal Patterns</a>
        <a href="#weather">Weather Impact</a>
        <a href="#vehicles">Vehicle Analysis</a>
        <a href="#casualties">Casualty Analysis</a>
        <a href="#findings">Key Findings</a>
    </div>
    
    <div class="container">
        <div id="overview" class="intro-section">
            <h1>UK Traffic Accident Analysis (2015-2018)</h1>
            
            <div class="key-stat">
                <p><strong>Analysis Overview:</strong></p>
                <ul>
                    <li>Total Accidents Analyzed: {total_accidents:,}</li>
                    <li>Total Casualties: {total_casualties:,}</li>
                    <li>Total Vehicles Involved: {total_vehicles:,}</li>
                    <li>Time Period: 2015-2018</li>
                </ul>
            </div>
        </div>

        <div id="datasets" class="visualization-section">
            <h2>Dataset Information</h2>
            
            <div class="dataset-section">
                <h3>Accidents Dataset</h3>
                <p>Contains detailed information about each accident:</p>
                <ul>
                    <li><code>Accident_Index</code>: Unique identifier for each accident</li>
                    <li><code>Date</code>, <code>Time</code>: When the accident occurred</li>
                    <li><code>Location</code>: Geographical coordinates and district information</li>
                    <li><code>Road_Type</code>, <code>Speed_limit</code>: Road characteristics</li>
                    <li><code>Weather_Conditions</code>, <code>Light_Conditions</code>: Environmental factors</li>
                </ul>
            </div>

            <div class="dataset-section">
                <h3>Casualties Dataset</h3>
                <p>Details about individuals involved in accidents:</p>
                <ul>
                    <li><code>Casualty_Reference</code>: Unique identifier for each casualty</li>
                    <li><code>Age_of_Casualty</code>, <code>Sex_of_Casualty</code>: Demographic information</li>
                    <li><code>Casualty_Type</code>: Type of road user (pedestrian, driver, etc.)</li>
                    <li><code>Casualty_Severity</code>: Severity of injuries</li>
                </ul>
            </div>

            <div class="dataset-section">
                <h3>Vehicles Dataset</h3>
                <p>Information about vehicles involved:</p>
                <ul>
                    <li><code>Vehicle_Reference</code>: Unique identifier for each vehicle</li>
                    <li><code>Vehicle_Type</code>: Category of vehicle</li>
                    <li><code>Age_of_Vehicle</code>, <code>Engine_Capacity_(CC)</code>: Vehicle characteristics</li>
                    <li><code>Age_of_Driver</code>, <code>Sex_of_Driver</code>: Driver information</li>
                </ul>
            </div>
        </div>

        <div id="severity" class="visualization-section">
            <h2>Accident Severity Analysis</h2>
            {plot_divs[0]}
            <div class="insight-box">
                <h3>Key Insights:</h3>
                <ul>
                    <li>Overall decrease in accident numbers from 2015 to 2018</li>
                    <li>Slight accidents form the majority but show the steepest decline</li>
                    <li>Fatal accidents remain relatively stable despite overall reductions</li>
                    <li>Serious accidents show a slight upward trend in proportion</li>
                </ul>
            </div>
        </div>

        <div id="temporal" class="visualization-section">
            <h2>Temporal Patterns</h2>
            {plot_divs[1]}
            <div class="insight-box">
                <h3>Time of Day Analysis:</h3>
                <ul>
                    <li>Peak accident times correlate with rush hours (8-9 AM and 5-6 PM)</li>
                    <li>Higher severity rates during nighttime hours</li>
                    <li>Lower accident frequency during early morning hours (2-5 AM)</li>
                    <li>Distinct patterns between weekdays and weekends</li>
                </ul>
            </div>
        </div>

        <div class="visualization-section">
            <h2>Age Distribution Analysis</h2>
            {plot_divs[2]}
            <div class="insight-box">
                <h3>Age-Related Findings:</h3>
                <ul>
                    <li>Young adults (18-25) show higher involvement in accidents</li>
                    <li>Elderly casualties (65+) tend to have more severe injuries</li>
                    <li>Children under 15 show distinct casualty patterns</li>
                    <li>Middle-aged adults show the most varied severity distribution</li>
                </ul>
            </div>
        </div>

        <div id="weather" class="visualization-section">
            <h2>Weather Impact Analysis</h2>
            {plot_divs[3]}
            <div class="insight-box">
                <h3>Weather-Related Insights:</h3>
                <ul>
                    <li>Fine weather accounts for majority of accidents due to higher traffic volume</li>
                    <li>Rain increases accident likelihood but not necessarily severity</li>
                    <li>Snow and ice show fewer but more severe accidents</li>
                    <li>Fog and mist conditions show higher severity rates</li>
                </ul>
            </div>
        </div>

        <div id="vehicles" class="visualization-section">
            <h2>Vehicle Type Analysis</h2>
            {plot_divs[4]}
            {plot_divs[5]}
            <div class="insight-box">
                <h3>Vehicle-Related Findings:</h3>
                <ul>
                    <li>Cars dominate accident statistics due to their prevalence</li>
                    <li>Motorcycles show disproportionately high severity rates</li>
                    <li>Heavy vehicles (trucks, buses) show lower frequency but higher severity</li>
                    <li>Road type significantly influences accident severity for different vehicles</li>
                </ul>
            </div>
        </div>

        <div id="casualties" class="visualization-section">
            <h2>Casualty Demographics</h2>
            {plot_divs[6]}
            <div class="insight-box">
                <h3>Demographic Insights:</h3>
                <ul>
                    <li>Gender disparities in accident involvement and severity</li>
                    <li>Age groups show different vulnerability patterns</li>
                    <li>Socio-economic factors influence casualty rates</li>
                    <li>Urban vs rural differences in casualty profiles</li>
                </ul>
            </div>
        </div>

        <div id="findings" class="visualization-section">
            <h2>Key Findings and Recommendations</h2>
            
            <h3>Major Trends</h3>
            <div class="key-stat">
                <ul>
                    <li>Overall accident rates show a declining trend (2015-2018)</li>
                    <li>Severity patterns vary significantly by time, location, and conditions</li>
                    <li>Demographic factors strongly influence accident outcomes</li>
                    <li>Environmental conditions play a crucial role in accident severity</li>
                </ul>
            </div>

            <h3>Recommendations</h3>
            <div class="insight-box">
                <ul>
                    <li>Enhanced safety measures during peak hours</li>
                    <li>Targeted interventions for high-risk age groups</li>
                    <li>Weather-specific traffic management strategies</li>
                    <li>Vehicle-specific safety campaigns</li>
                    <li>Improved road design at high-risk locations</li>
                </ul>
            </div>

            <h3>Future Research Directions</h3>
            <div class="key-stat">
                <ul>
                    <li>Deep dive into specific vehicle type patterns</li>
                    <li>Analysis of intervention effectiveness</li>
                    <li>Seasonal pattern investigation</li>
                    <li>Socio-economic factor analysis</li>
                </ul>
            </div>
        </div>
    </div>
</body>
</html>
'''

# Write the HTML file
with open('visualizations/comprehensive_analysis.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

print("Analysis complete! Open 'visualizations/comprehensive_analysis.html' in a web browser to view the complete analysis.")

Analysis complete! Open 'visualizations/comprehensive_analysis.html' in a web browser to view the complete analysis.
